<a href="https://colab.research.google.com/github/snow-The/GW190521/blob/main/homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 依賴安裝
!pip install "ml4gw>=0.7.10" "gwpy>=3.0" "h5py>=3.12" "torchmetrics>=1.6" "lightning>=2.4.0" "rich>=10.2.2,<14.0"

In [ ]:
import torch
import numpy as np
from gwpy.timeseries import TimeSeries
import matplotlib.pyplot as plt

# 1. 設定設備
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# 2. 設定 GW190521 (v4) 參數
trigger_time = 1242442967.4
sample_rate = 2048
start_time = trigger_time - 6
end_time = trigger_time + 2
ifos = ["H1", "L1"]

# 3. 下載並檢查數據
data_ts = []
print("Downloading data...")

try:
    for det in ifos:
        # 下載
        data = TimeSeries.fetch_open_data(det, start_time, end_time, verbose=True)
        # 重取樣
        data = data.resample(sample_rate)

        # === 安全檢查 1: 檢查是否有 NaN (空值) ===
        if np.isnan(data.value).any():
            print(f"⚠️ 警告: {det} 數據包含 NaN，正在填補...")
            # 簡單填補，避免 GPU 崩潰 (雖然通常 NaN 不會導致 Assert，但保險起見)
            data.value = np.nan_to_num(data.value)

        # 轉成 Tensor (還在 CPU)
        data_ts.append(torch.from_numpy(data.value.copy()).float()) # 確保是 float32

    # === 安全檢查 2: 檢查形狀是否一致 ===
    if data_ts[0].shape != data_ts[1].shape:
        print(f"⚠️ 形狀不匹配: H1={data_ts[0].shape}, L1={data_ts[1].shape}")
        # 強制修剪到較小的長度
        min_len = min(data_ts[0].shape[0], data_ts[1].shape[0])
        data_ts[0] = data_ts[0][:min_len]
        data_ts[1] = data_ts[1][:min_len]

    # 4. 堆疊並搬移到 GPU (這是原本報錯的地方)
    # 先 stack 成 (Batch, Channel, Time)
    data_tensor = torch.stack(data_ts).unsqueeze(0)

    # 最後才搬去 GPU
    data_tensor = data_tensor.to(device)

    print("✅ 數據成功載入 GPU！")
    print(f"Data shape: {data_tensor.shape}")

except Exception as e:
    print(f"❌ 發生錯誤: {e}")
    print("請嘗試重啟 Runtime。")

In [ ]:
# 初步處理數據
from ml4gw.transforms import Whiten, SpectralDensity

# 1. 獲取背景數據來計算 PSD
# 我們往前多抓 64 秒的數據來估算雜訊分佈
psd_duration = 64
psd_start = start_time - psd_duration
psd_end = start_time

psd_data_ts = []
print("Downloading background data for PSD...")
for det in ifos:
    psd_data = TimeSeries.fetch_open_data(det, psd_start, psd_end, verbose=False)
    psd_data = psd_data.resample(sample_rate)
    psd_data_ts.append(torch.from_numpy(psd_data.value.copy()))

psd_tensor = torch.stack(psd_data_ts).to(device)

# 2. 定義 ml4gw 的處理模組
# 這些模組可以放在 GPU 上加速運算
fftlength = 2
spectral_density = SpectralDensity(
    sample_rate=sample_rate,
    fftlength=fftlength,
    average="median", # 使用中位數平均法來抗干擾
).to(device)

whiten = Whiten(
    fduration=2,       # 白化時會切掉邊緣各 1 秒
    sample_rate=sample_rate,
    highpass=20        # 同時做 Highpass filter (20Hz)
).to(device)

# 3. 執行計算
# 計算 PSD (記得轉成 double 精度以防溢位)
psd = spectral_density(psd_tensor.double())

# 執行白化 (Whitening)
# 注意：Whiten 會自動幫你做 Bandpass (highpass) 和除以 PSD
whitened_data = whiten(data_tensor, psd)

print(f"Whitened data shape: {whitened_data.shape}")

In [ ]:
# === 修正後的 Q-transform ===
# 必須指定 frange (頻率範圍)，避免 log(0) 錯誤
q_transform = SingleQTransform(
    duration=duration,
    sample_rate=sample_rate,
    spectrogram_shape=(512, 1024),
    q=10,
    frange=(20, sample_rate/2) # <--- 關鍵修正：明確指定從 20Hz 開始
).to(device)

# 計算 Spectrogram
# 為了保險，我們先檢查輸入數據有沒有 NaN (非數值)
if torch.isnan(whitened_data).any():
    print("警告: 數據中包含 NaN，請檢查 Whitening 步驟！")
else:
    specgram = q_transform(whitened_data)
    print("Spectrogram 計算成功！")

    # === 繪圖 ===
    import numpy as np

    # 把數據轉回 CPU
    spec_np = specgram[0, 1].cpu().numpy() # 取 L1 偵測器

    plt.figure(figsize=(12, 8))
    # 注意 extent 的 Y 軸要改成我們設定的 frange
    plt.imshow(spec_np, aspect="auto", origin="lower",
               extent=[start_time, end_time, 20, sample_rate/2])
    plt.yscale('log')
    plt.ylim(20, 150)
    plt.xlim(trigger_time - 0.5, trigger_time + 0.2)
    plt.xlabel("GPS Time (s)")
    plt.ylabel("Frequency (Hz)")
    plt.colorbar(label="Normalized Energy")
    plt.title("GW190521 Spectrogram (L1) - Corrected")
    plt.show()